***Smartcab Project Report ***
==============================

by Moritz Bendiek






Implement a basic driving agent
-------------------------------

The requested behavior (random choice of action disregarding any input information) is realized by adding the following code to the `update` method:
```python
action_list = [None, 'forward', 'left', 'right']
action = action_list[random.randint(0,3)]
```

Note that `random` should be initialized (via `seed`) first.

**QUESTION:** Observe what you see with the agent's behavior as it takes random actions. Does the **smartcab** eventually make it to the destination? Are there any other interesting observations to note?

**ANSWER**: The agent executes a "random walk". Due to the limited size of the world model (6*8=48 positions on the grid), the smartcab will arrive at its destination within a reasonably short time span. I executed 100 test runs and found that the smartcab arrived at the destination before the hard time limit (-100) was reached 63 times, and 37 times the agent ran into the hard time limit.

The smartcab apparently is influenced by the traffic lights, insofar as it does not always move when it sees a "red" light. However, judging from the visualization, I am not sure that this behavior is consistent. Also, the movement of the smartcab somedoes does not seem to fit to the chosen action (e.g. displays 'left' as action, but drives forward). 

Inform the driving agent
------------------------

**TASK:** Now that your driving agent is capable of moving around in the environment, your next task is to identify a set of states that are appropriate for modeling the smartcab and environment. The main source of state variables are the current inputs at the intersection, but not all may require representation. You may choose to explicitly define states, or use some combination of inputs as an implicit state. At each time step, process the inputs and update the agent's current state using the `self.state` variable. Continue with the simulation deadline enforcement `enforce_deadline` being set to `False`, and observe how your driving agent now reports the change in state as the simulation progresses.

**QUESTION:** What states have you identified that are appropriate for modeling the smartcab and environment? Why do you believe each of these states to be appropriate for this problem?

**ANSWER:** While intuitively the position of the smartcab on the grid should be part of its state, I think I can omit it as in order to learn to drive it is not relevant for the smartcab where it is, as all traffic rules apply equally in all places.

To differentiate between situations that require different behavior of the agent, it is sufficient to regard a few sub-states defined based on the inputs / destination:
* In which direction shall the smartcab move? Here, we can use `self.next_waypoint` directly. It can be any of `['left', 'forward', 'right']`
* Is the smartcab allowed to move left? This is either `True` or `False` and can be determined according to the know traffic rules: `True` if `light` is `green` and `oncoming` is either `forward` of `left`, else `False`.
* Is the smartcab allowed to move forward? This is `True` if `light` is `green`, else it is `False`.
* Is the smartcab allowed to move right? This is `False` if `left` is `forward` and else `True`.

Therefore I choose to model the state as follows:
``` python
left_allowed = (inputs['light']=='green') and (inputs['oncoming'] in ['left', None])
forward_allowed = inputs['light']=='green'
right_allowed = (inputs['light']=='green') or (inputs['left'] in [None, 'left', 'right'])
self.state = [self.next_waypoint, left_allowed, forward_allowed, right_allowed]
```


**OPTIONAL:** How many states in total exist for the smartcab in this environment? Does this number seem reasonable given that the goal of Q-Learning is to learn and make informed decisions about each state? Why or why not?

**ANSWER:** The total number of states is the Cartesian Product of all the dimensions, in this case 3 \* 2 \* 2 \* 2 = 24 states.

The goal of Q-Learning is to move through all states infinitely (read: *very*) often. As in the model there are 100 test runs, and each test run consists of a capped number of individual moves (even with `enforce_deadline` being set to `False`, a run stops after hitting the hard deadline of -100), we can broadly assume that with 100 moves per run, we will visit 10000 states. However it is likely that the probability to be in a specific state can vary (i.e. is not uniform).  

Note: Not all combinations are possible in practice! E.g. whenever `forward_allowed` is True, `right_allowed` is always True as well. 


Implement a Q-Learning Driving Agent
------------------------------------

**TASK:** With your driving agent being capable of interpreting the input information and having a mapping of environmental states, your next task is to implement the Q-Learning algorithm for your driving agent to choose the best action at each time step, based on the Q-values for the current state and action. Each action taken by the smartcab will produce a reward which depends on the state of the environment. The Q-Learning driving agent will need to consider these rewards when updating the Q-values. Once implemented, set the simulation deadline enforcement `enforce_deadline` to `True`. Run the simulation and observe how the smartcab moves about the environment in each trial.

**QUESTION:** What changes do you notice in the agent's behavior when compared to the basic driving agent when random actions were always taken? Why is this behavior occurring?

**ANSWER: ** Out of 100 test runs (alpha = 0.1, gamma = 0.2, epsilon = 0.05) the smartcab reached its destination in *some* of the cases. The result is very volatile between sets of runs. While I find on average (based on 21 sets of runs) the smartcab reaches its destination in 49% of the runs, the percentage of successful runs in a set varies between 4% and 93%, with a standard deviation of 29%. This inconsistency is somewhat disappointing.

When looking at the change of behavior within one set of 100 test runs, it becomes apparent that zero rewards are apparently common in early runs, while the become rarer in later runs. This behavior is consistent with the notion that the Q-Matrix is initialized with a uniform value which in early stages effects a more or less random choice of actions, which often have a zero reward, whereas in later stages enough learning has happened to pick positively rewarded actions more frequently.

As I analyzed 21 sets of tests runs, I found that in the first 50 runs (of each set) the average success rate is ~40%, whereas in the last 50 runs the average success rate is ~60%.


Improve the Q-Learning Driving Agent
------------------------------------

Your final task for this project is to enhance your driving agent so that, after sufficient training, the smartcab is able to reach the destination within the allotted time safely and efficiently. Parameters in the Q-Learning algorithm, such as the learning rate (`alpha`), the discount factor (`gamma`) and the exploration rate (`epsilon`) all contribute to the driving agent’s ability to learn the best action for each state. To improve on the success of your smartcab:

* Set the number of trials, `n_trials`, in the simulation to 100.
* Run the simulation with the deadline enforcement `enforce_deadline` set to `True` (you will need to reduce the update delay `update_delay` and set the `display` to `False`).
* Observe the driving agent’s learning and smartcab’s success rate, particularly during the later trials.
* Adjust one or several of the above parameters and iterate this process.

This task is complete once you have arrived at what you determine is the best combination of parameters required for your driving agent to learn successfully.

**QUESTION:** Report the different values for the parameters tuned in your basic implementation of Q-Learning. For which set of parameters does the agent perform best? How well does the final driving agent perform?


**ANSWER: ** The tuning for the parameters `alpha, gamma, epsilon` is done by writing some loops around the exisiting code in the `run()` method:

```python
for alpha in [0.05,0.1,0.2,0.4]:
        for gamma in [0.05,0.1,0.2,0.4]:
            for epsilon in [0.025,0.05]:
```
In addition, to obtain some meaningful statistics, every combination is repeated 100 times (so for any given hyper-parameter combination, there are 100 * 100 test runs).

The analysis was done by writing the output of `agent.py` to a file a using a quick-and-dirty python script to rate of successul trials for each hyper-paramter combination. For the result, only the last 10 of 100 runs was used. The assumption is, that after 90 runs, the model has learned enough to achieve good results.

The result is shown below. Note that the maximum for reached appears with `alpha=0.05; epsilon=0.025; gamma=0.4` and is only 53.9% success rate.

|    |alpha  |epsilon  |failed | gamma | reached|
|---   | ---  | ---   |  --- | ---  |   --- |
|1   | 0.05  | 0.025   | 60.6  | 0.05  |   39.4 |
|2   | 0.05  | 0.050   | 48.3  | 0.05  |   51.7 |
|3   | 0.05  | 0.025   | 67.3  | 0.10  |   32.7 |
|4   | 0.05  | 0.050   | 57.0  | 0.10  |   43.0 |
|5   | 0.05  | 0.025   | 51.4  | 0.20  |   48.6 |
|6   | 0.05  | 0.050   | 48.9  | 0.20  |   51.1 |
|7   | 0.05  | 0.025   | 46.1  | 0.40  |   53.9 |
|8   | 0.05  | 0.050   | 47.9  | 0.40  |   52.1 |
|9   | 0.10  | 0.025   | 61.0  | 0.05  |   39.0 |
|10  | 0.10  | 0.050   | 53.0  | 0.05  |   47.0 |
|11  | 0.10  | 0.025   | 61.5  | 0.10  |   38.5 |
|12  | 0.10  | 0.050   | 67.6  | 0.10  |   32.4 |
|13  | 0.10  | 0.025   | 57.2  | 0.20  |   42.8 |
|14  | 0.10  | 0.050   | 56.7  | 0.20  |   43.3 |
|15  | 0.10  | 0.025   | 47.9  | 0.40  |   52.1 |
|16  | 0.10  | 0.050   | 47.6  | 0.40  |   52.4 |
|17  | 0.20  | 0.025   | 62.8  | 0.05  |   37.2 |
|18  | 0.20  | 0.050   | 62.1  | 0.05  |   37.9 |
|19  | 0.20  | 0.025   | 57.7  | 0.10  |   42.3 |
|20  | 0.20  | 0.050   | 62.2  | 0.10  |   37.8 |
|21  | 0.20  | 0.025   | 60.8  | 0.20  |   39.2 |
|22  | 0.20  | 0.050   | 64.5  | 0.20  |   35.5 |
|23  | 0.20  | 0.025   | 52.4  | 0.40  |   47.6 |
|24  | 0.20  | 0.050   | 61.1  | 0.40  |   38.9 |
|25  | 0.40  | 0.025   | 68.8  | 0.05  |   31.2 |
|26  | 0.40  | 0.050   | 75.2  | 0.05  |   24.8 |
|27  | 0.40  | 0.025   | 78.3  | 0.10  |   21.7 |
|28  | 0.40  | 0.050   | 74.9  | 0.10  |   25.1 |
|29  | 0.40  | 0.025   | 68.3  | 0.20  |   31.7 |
|30  | 0.40  | 0.050   | 75.7  | 0.20  |   24.3 |
|31  | 0.40  | 0.025   | 59.9  | 0.40  |   40.1 |




**QUESTION:** Does your agent get close to finding an optimal policy, i.e. reach the destination in the minimum possible time, and not incur any penalties? How would you describe an optimal policy for this problem?

**ANSWER:** The agent does *not* get close to finding an optimal policy. The general success rate should be close to 100% and the occurrence of negative rewards should be close to zero. Both does not hold true, which suggests a major flaw somewhere in my model.

An optimal policy should be to move in a direction which reduces the distance to the target and to obey all traffic rules (i.e. don't move when not allowed to move in *any* direction that reduces distance to target.)